In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import keras
import keras.backend as K
from keras.layers import Flatten
from keras.datasets import imdb

from keras.preprocessing import sequence

from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

Using TensorFlow backend.


In [2]:
vocabulary_size = 6000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size, skip_top=127, oov_char= 00 )

print('Loaded dataset with {} training samples, {} test samples'.format(len(X_train), len(X_test)))
print('---review---')
print(X_train[6])
print('---label---')
print(y_train[6])

print('Maximum review length: {}'.format(
len(max((X_train + X_test), key=len))))

print('Minimum review length: {}'.format(
len(min((X_train + X_test), key=len))))

17465344/17464789 [==============================] - 0s 0us/step
Loaded dataset with 25000 training samples, 25000 test samples
---review---
[0, 0, 365, 1234, 0, 1156, 354, 0, 0, 5327, 0, 0, 1016, 0, 5940, 356, 0, 0, 1349, 500, 746, 0, 200, 0, 4132, 0, 0, 0, 1117, 1831, 0, 0, 4831, 0, 0, 0, 4183, 0, 369, 0, 215, 1345, 143, 0, 0, 1838, 0, 1974, 0, 0, 0, 257, 0, 0, 486, 0, 0, 0, 0, 0, 271, 0, 196, 0, 949, 4121, 0, 0, 0, 0, 2212, 2436, 819, 0, 0, 0, 0, 180, 0, 227, 0, 0, 2494, 0, 0, 423, 0, 168, 0, 0, 0, 0, 0, 665, 0, 270, 0, 0, 0, 197, 0, 161, 5390, 0, 0, 0, 0, 0, 419, 665, 0, 0, 0, 0, 0, 0, 2084, 0, 4773, 0, 0, 0, 1901]
---label---
1
Maximum review length: 2697
Minimum review length: 70


In [3]:
max_words = 2315
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

print('shape train:',X_train.shape, 'shape test:', X_test.shape)

shape train: (25000, 2315) shape test: (25000, 2315)


In [4]:
embedding_size= 64
model=Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=X_train.shape[1]))

model.add(LSTM(60, return_sequences= True))
model.add(LSTM(40, return_sequences= True))
model.add(LSTM(20, return_sequences= True))
model.add(Flatten())

model.add(Dense(1, activation='relu'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2315, 64)          384000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 2315, 60)          30000     
_________________________________________________________________
lstm_2 (LSTM)                (None, 2315, 40)          16160     
_________________________________________________________________
lstm_3 (LSTM)                (None, 2315, 20)          4880      
_________________________________________________________________
flatten_1 (Flatten)          (None, 46300)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 46301     
Total params: 481,341
Trainable params: 481,341
Non-trainable params: 0
________________________________________________

In [6]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
			 
batch_size = 512
num_epochs = 10
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs)
scores = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', scores[1])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 24488 samples, validate on 512 samples
Epoch 1/10
24488/24488 [==============================] - 1540s 63ms/step - loss: 7.7125 - accuracy: 0.5000 - val_loss: 7.7125 - val_accuracy: 0.5000
Epoch 2/10
24488/24488 [==============================] - 1553s 63ms/step - loss: 7.7125 - accuracy: 0.5000 - val_loss: 7.7125 - val_accuracy: 0.5000
Epoch 3/10
24488/24488 [==============================] - 1555s 64ms/step - loss: 7.7125 - accuracy: 0.5000 - val_loss: 7.7125 - val_accuracy: 0.5000
Epoch 4/10
24488/24488 [==============================] - 1541s 63ms/step - loss: 7.7125 - accuracy: 0.5000 - val_loss: 7.7125 - val_accuracy: 0.5000
Epoch 5/10
24488/24488 [==============================] - 1534s 63ms/step - loss: 7.7125 - accuracy: 0.5000 - val_loss: 7.7125 - val_accuracy: 0.5000
Epoch 6/10
24488/24488 [==============================] - 1545s 63ms/step - loss: 7.7125 - accuracy: 0.5000 - val_loss: 7.7125 - val_accuracy: 0.5000
Epoch 7/10
10752/24488 [============>...............

KeyboardInterrupt: ignored